# Build Dataset with or without background 
_PP-HumanSeg_  
_https://github.com/PaddlePaddle/PaddleSeg/tree/release/2.6/contrib/PP-HumanSeg_

# Set up

In [ ]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-f1a6a2ae-7da6-732c-b007-55fc95a88e48)


In [ ]:
%%capture
!python -m pip install paddlepaddle-gpu==2.3.2.post111 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
%cd /content
!git clone https://github.com/jujemu/PaddleSeg
%cd /content/PaddleSeg
!pip install -r requirements.txt

In [ ]:
%%capture
%cd /content/PaddleSeg/contrib/PP-HumanSeg
!python src/download_inference_models.py
!python src/download_data.py

In [ ]:
import os
import glob
import shutil
import re

from tqdm import tqdm
from PIL import Image

## Base Image

 - save directory: /content/input_with_background
 - Set the brand

[ 'Christian Dior', 'Maison Margiela', 'Dries Van Noten'  'Miu Miu']

In [ ]:
# Base Dataset
# save directory: "/content/base_input"
f_name = '/content/base_input'
!rm -rf $f_name
!mkdir -p $f_name
%cd $f_name
SIZE = (512, 1024)

path = '/content/drive/MyDrive/Colab Notebooks/AiffelThon/Collection/Paris/'
#------------------------------------------------------
# Set the brand
brand = 'Saint Laurent'
#------------------------------------------------------
brand_ = brand.replace(" ", "")
input = os.path.join(path, brand)

cnt = 0
gap = len(list(glob.iglob(input+'/**/Look_*.png', recursive=True)))
for img_path, _ in zip(glob.iglob(input+'/**/Look_*.png', recursive=True), tqdm(range(gap))):
  img = Image.open(img_path)
  img = img.resize((SIZE[0], img.size[1]), Image.BICUBIC)
  img = img.resize(SIZE, Image.BICUBIC)
  
  img_name = brand_ + '_' + img_path.split('/')[-1]
  n = re.sub('([0-9]+)', str(cnt).zfill(5), img_name)
  cnt += 1
  img.save(n)
print()

# Image with background
# (width, height) 512, 512
# save directory: "/content/input_with_background"
f_name = "/content/input_with_background"
!rm -rf $f_name
!mkdir -p $f_name
%cd $f_name
SIZE = (512, 512)

list_img_path = os.listdir('/content/base_input')
for img_path in tqdm(list_img_path):
  path = os.path.relpath('/content/base_input/' + img_path)
  img = Image.open(path)
  img = img.resize(SIZE, Image.BICUBIC)
  img.save(img_path)
print()

# Image without background
# Applied segmentation
# (width, height) 512, 1024
# save directory: "/content/input_without_background"
f_name = "/content/input_without_background"
!rm -rf $f_name
!mkdir -p $f_name
%cd /content/PaddleSeg/contrib/PP-HumanSeg

print('Image without background')
!python src/predict.py \
  --config inference_models/portrait_pp_humansegv2_lite_256x144_inference_model_with_softmax/deploy.yaml \
  --img_path /content/base_input/ \
  --save_dir /content/input_without_background

/content/base_input


100%|█████████▉| 5370/5371 [58:35<00:00,  1.53it/s]



/content/input_with_background


100%|██████████| 5371/5371 [11:27<00:00,  7.81it/s]



/content/PaddleSeg/contrib/PP-HumanSeg
Image without background
2022-08-31 15:16:50 [INFO]	Input: image
2022-08-31 15:16:50 [INFO]	Create predictor...
2022-08-31 15:16:51 [INFO]	Start predicting...


In [ ]:
%cd /content
input_1 = brand_ + '_input_with_background.zip'
input_2 = brand_ + '_input_without_background.zip'

!zip -rq $input_1 input_with_background/
!zip -rq $input_2 input_without_background/
!mv $input_1 '/content/drive/MyDrive/Colab Notebooks/AiffelThon/jemu/dataset'
!mv $input_2 '/content/drive/MyDrive/Colab Notebooks/AiffelThon/jemu/dataset'

/content
